In [7]:
# Handle table-like data and matrices
import numpy as np
import pandas as pd

# Helpers
import sys
from datetime import datetime
import pickle

from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.metrics import accuracy_score,roc_auc_score,roc_curve,log_loss
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,cross_validate
from sklearn.metrics import classification_report
from sklearn.model_selection import LeaveOneOut,KFold,StratifiedKFold
from sklearn.decomposition import PCA,KernelPCA
import lightgbm as lgb

import sys
sys.path.insert(0,'../../') 
from helpers import *
from MAP_estimator import MAP_estimator

# Algorithm
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

data_path = '../data/'

In [8]:
count_df = pd.read_pickle(data_path+'count_df_with_label.pkl')
bin_df = pd.read_pickle(data_path+'bin_df.pkl')
freq_df = pd.read_pickle(data_path+'freq_df.pkl')
train_y = pd.read_csv(data_path+'train_y.csv')

In [9]:
# count_df = count_df.merge(train_y,on='sample_name')
bin_df = bin_df.merge(train_y,on='sample_name')
freq_df = freq_df.merge(train_y,on='sample_name')

In [10]:
clf = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                        learning_rate=0.1, max_depth=-1, min_child_samples=10,
                        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
                        n_jobs=-1, num_leaves=31, objective='binary', random_state=0,
                        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
                        subsample_for_bin=200000, subsample_freq=1)

In [ ]:
#     for tcr in asso_TCRs:
#         if tcr not in features:
#             features[tcr] = 1
#         else:
#             features[tcr] += 1
#     T = pd.DataFrame(features,index=[0]).T.sort_values(by=0, ascending=False)
#     asso_TCRs = T.iloc[:50].T.columns.values.tolist()

In [47]:
# Init lists
# auroc = []
# kf = StratifiedKFold(n_splits=10)
# for train_index,test_index in kf.split(count_df,count_df['phenotype_status']):
#     train_cv, test_cv = count_df.iloc[train_index], count_df.iloc[test_index]
#     train_neg = train_cv[train_cv['phenotype_status']==0]
#     train_pos = train_cv[train_cv['phenotype_status']==1]
#     asso_TCRs = TCRs_selection_Fisher(train_neg,train_pos,TCRs,0.2)
#     features = {}
#     for tcr in asso_TCRs:
#         if tcr not in features:
#             features[tcr] = 1
#         else:
#             features[tcr] += 1
#     T = pd.DataFrame(features,index=[0]).T.sort_values(by=0, ascending=False)
#     asso_TCRs = T.iloc[:100].T.columns.values.tolist()
#     clf.fit(train_cv[asso_TCRs],train_cv['phenotype_status'])
#     test_prob = clf.predict_proba(test_cv[asso_TCRs])[:,1]
#     auc = roc_auc_score(test_cv['phenotype_status'],test_prob)
#     auroc.append(auc)

In [ ]:
# # Init lists
# features = {}

# kf1 = StratifiedKFold(n_splits=5)
# for train_index,test_index in kf.split(count_df,count_df['sample_name']):
#     train_cv, test_cv = data.iloc[train_index], data.iloc[test_index]
#     kf2 = StratifiedKFold(n_splits=5):
#         train_cv, test_cv = count_df.iloc[train_index], count_df.iloc[test_index]
#         train_neg = train_cv[train_cv['phenotype_status']==0]
#         train_pos = train_cv[train_cv['phenotype_status']==1]
#         asso_TCRs = TCRs_selection_Fisher(train_neg,train_pos,TCRs,0.2)
#         for tcr in asso_TCRs:
#             if tcr not in features:
#                 features[tcr] = 1
#             else:
#                 features[tcr] += 1

# print('cv auroc:',roc_auc_score(y_true,y_proba))
# print('cv log_loss:',log_loss(y_true,y_proba))

In [11]:
clf = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                        learning_rate=0.1, max_depth=-1, min_child_samples=30,min_data=1,
                        min_child_weight=0.001, min_split_gain=0.0, n_estimators=200,
                        n_jobs=-1, num_leaves=28, objective='binary', random_state=0,
                        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
                        subsample_for_bin=200000, subsample_freq=1)
# Init lists
sample_name = []
y_true = []
y_pred = []
y_proba = [] 

TCRs = count_df.drop(['sample_name','phenotype_status'],axis=1).columns.values
print(clf)
kf = LeaveOneOut()
for train_index,test_index in kf.split(count_df):
    data = count_df.copy(deep=True)
    train_cv, test_cv = data.iloc[train_index], data.iloc[test_index]
    print(test_cv['sample_name'].tolist()[0])
    
    clf.fit(train_cv[TCRs],train_cv['phenotype_status'])
    test_pred = clf.predict(test_cv[TCRs])[0]
    test_prob = clf.predict_proba(test_cv[TCRs])[:,1][0]

    sample_name.append(test_cv['sample_name'].tolist()[0])
    y_true.append(test_cv['phenotype_status'].values[0])
    y_pred.append(test_pred)
    y_proba.append(test_prob)

    
    print('y_true:',test_cv['phenotype_status'].values[0],'y_pred:',test_pred,'posterior_c1:',test_prob)
    print()

print('cv auroc:',roc_auc_score(y_true,y_proba))
print('cv log_loss:',log_loss(y_true,y_proba))

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=30,
        min_child_weight=0.001, min_data=1, min_split_gain=0.0,
        n_estimators=200, n_jobs=-1, num_leaves=28, objective='binary',
        random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=1)
RA47
y_true: 1 y_pred: 1 posterior_c1: 0.9942040898024993

HC9
y_true: 0 y_pred: 1 posterior_c1: 0.999649082034503

RA29
y_true: 1 y_pred: 0 posterior_c1: 0.0005473890638540823

RA8
y_true: 1 y_pred: 1 posterior_c1: 0.9999965286851477

RA63
y_true: 1 y_pred: 1 posterior_c1: 0.9999953452564091

RA33
y_true: 1 y_pred: 1 posterior_c1: 0.9994779456001506

HC17
y_true: 0 y_pred: 1 posterior_c1: 0.9953947222439401

HC3
y_true: 0 y_pred: 1 posterior_c1: 0.9999940264421455

RA17
y_true: 1 y_pred: 1 posterior_c1: 0.9999944255005938

RA46
y_true: 1 y_pred: 1 posterior_c1: 0.9926655019653507
